In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
from shapely.geometry import Point
import os

In [2]:
def nearest_road_distance_angle(name, geojson_path, lon_series, lat_series):
    import numpy as np
    import pandas as pd
    import geopandas as gpd
    from shapely.geometry import Point, MultiLineString

    # Load roads (only linear geometries)
    roads = gpd.read_file(geojson_path)
    roads = roads[roads.geometry.type.isin(["LineString", "MultiLineString"])].copy()
    roads = roads.to_crs("EPSG:4326")

    # Points
    points = gpd.GeoDataFrame(
        {"lon": lon_series, "lat": lat_series},
        geometry=gpd.points_from_xy(lon_series, lat_series),
        crs="EPSG:4326"
    )

    # UTM zone (choose hemisphere by mean latitude)
    mean_lon = float(points["lon"].mean())
    mean_lat = float(points["lat"].mean())
    utm_zone = int((mean_lon + 180) / 6) + 1
    epsg_utm = (32600 if mean_lat >= 0 else 32700) + utm_zone

    points_utm = points.to_crs(epsg_utm)
    roads_utm  = roads.to_crs(epsg_utm)

    # Nearest spatial join
    nearest = gpd.sjoin_nearest(points_utm, roads_utm, how="left", distance_col="distance_m")

    # Pull the matched road geometry using index_right (this avoids needing a road_geom column)
    # Reindex aligns NaNs for unmatched rows automatically.
    geom_point = nearest.geometry
    geom_road  = roads_utm.geometry.reindex(nearest["index_right"]).to_numpy()

    # For each point, compute the closest point on its matched road
    nearest_pts = []
    for p, r in zip(geom_point, geom_road):
        if r is None or (getattr(r, "is_empty", False)):
            nearest_pts.append(Point(np.nan, np.nan))
            continue
        try:
            if isinstance(r, MultiLineString):
                r = min(list(r.geoms), key=lambda part: part.distance(p))
            nearest_pts.append(r.interpolate(r.project(p)))
        except Exception:
            nearest_pts.append(Point(np.nan, np.nan))

    nearest_pts = gpd.GeoSeries(nearest_pts, crs=epsg_utm)

    # dx, dy, angle (east=0°, CCW); normalize to [0, 360)
    dx = nearest_pts.x.values - geom_point.x.values
    dy = nearest_pts.y.values - geom_point.y.values
    angle_deg = (np.degrees(np.arctan2(dy, dx)) + 360) % 360

    # Debug
    '''print("🧭 Diagnostics:")
    print(f"  • Total points: {len(points)}")
    print(f"  • Missing joined roads: {nearest['index_right'].isna().sum()}")
    print(f"  • Failed nearest points: {np.isnan(dx).sum()}")
    print(f"  • CRS used: EPSG:{epsg_utm}")'''

    n = len(points)
    return pd.DataFrame({
        name+"distance_km": nearest["distance_m"].values[:n]/1000,
        name+"angle_deg":  angle_deg[:n],
    })


In [3]:
folder = "./MAIAC AOD gap filled nc/"
paths = os.listdir(folder)
aod = xr.concat([xr.open_dataset(folder + path) for path in paths], dim='time').to_dataframe().reset_index()

c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
aod.drop('filled_AOD', axis=1, inplace=True)

In [5]:
aod.sort_values(['time', 'latitude', 'longitude'], ascending=[True, False, True], inplace=True)
aod.reset_index(inplace=True, drop=True)

In [6]:
lats = aod['latitude'].unique()
lons = aod['longitude'].unique()
times = aod['time'].unique()

In [7]:
121*191

23111

In [8]:
aod[:23111]['latitude']

0        32.2
1        32.2
2        32.2
3        32.2
4        32.2
         ... 
23106    31.0
23107    31.0
23108    31.0
23109    31.0
23110    31.0
Name: latitude, Length: 23111, dtype: float64

In [9]:
len(aod)

34735833

In [10]:
types = ['Motorway ', 'Primary ', 'Secondary ', 'Trunk ']
folder = './Amman roads/'
ext = 'roads Amman.geojson'

In [11]:
dfs = []
for type in types:
    dfs.append(nearest_road_distance_angle(type, folder + type +'roads Amman.geojson', 
                                           aod[:23111]['longitude'], aod[:23111]['latitude']))

In [12]:
df = pd.concat(dfs, axis=1)

In [13]:
df

,Motorway distance_km,Motorway angle_deg,Primary distance_km,Primary angle_deg,Secondary distance_km,Secondary angle_deg,Trunk distance_km,Trunk angle_deg
0,42.842927,320.157155,12.278472,325.644368,8.121893,275.646062,4.066180,189.937230
1,42.842927,320.157155,11.493624,323.116327,8.053082,268.973963,3.253366,337.321779
2,42.103025,319.366803,10.734457,320.224007,8.094560,262.279274,2.347855,345.454674
3,42.103025,319.366803,10.006815,316.901124,8.244659,255.740946,1.426460,346.785526
4,41.371481,318.548250,9.318084,313.072071,7.783866,354.854711,0.501235,346.785281
...,...,...,...,...,...,...,...,...
23106,62.627359,113.838305,110.848451,166.741315,12.917368,99.625694,22.977185,337.022297
23107,62.627359,113.838305,111.781593,166.845695,13.129633,103.716368,22.092015,336.109322
23108,63.040265,114.621900,112.715078,166.948389,13.406428,107.657651,21.212961,335.120201
23109,63.040265,114.621900,113.648897,167.049437,13.743853,111.422633,20.340816,334.045699


In [18]:

for col in df.columns:
    aod[col] = np.float32(np.kron(df[col].values.reshape(len(lats), len(lons)), np.ones((len(times), 1, 1)))).flatten()


In [19]:
dat = aod.set_index(['time', 'latitude', 'longitude']).to_xarray()

In [20]:
dat

<xarray.Dataset>
Dimensions:                (time: 1503, latitude: 121, longitude: 191)
Coordinates:
  * time                   (time) datetime64[ns] 2020-12-10 ... 2025-01-20
  * latitude               (latitude) float64 31.0 31.01 31.02 ... 32.19 32.2
  * longitude              (longitude) float64 35.5 35.51 35.52 ... 37.39 37.4
Data variables:
    Motorway distance_km   (time, latitude, longitude) float32 29.43 ... 42.81
    Motorway angle_deg     (time, latitude, longitude) float32 45.26 ... 229.7
    Primary distance_km    (time, latitude, longitude) float32 115.3 ... 112.4
    Primary angle_deg      (time, latitude, longitude) float32 167.8 ... 183.4
    Secondary distance_km  (time, latitude, longitude) float32 14.15 ... 89.18
    Secondary angle_deg    (time, latitude, longitude) float32 123.9 ... 262.8
    Trunk distance_km      (time, latitude, longitude) float32 15.96 ... 18.61
    Trunk angle_deg        (time, latitude, longitude) float32 321.8 ... 172.4

In [21]:
# Number of chunks
n = 32

# Get indices to split along the time dimension
time_len = dat.dims["time"]
splits = np.array_split(np.arange(time_len), n)

# Save each chunk to a separate file
for i, idx in enumerate(splits):
    ds_subset = dat.isel(time=idx)
    ds_subset.to_netcdf(f"./Roads nc/Roads_{i+1}.nc")